# BMI/CS 576 Fall 2019 - HW2
The objectives of this homework are to practice

* implementing sequence alignment algorithms
* devising dynamic programming algorithms
* working with probabilistic models

## HW policies
Before starting this homework, please read over the [homework policies](https://canvas.wisc.edu/courses/167969/pages/hw-policies) for this course.  In particular, note that homeworks are to be completed *individually*.

You are welcome to use any code from the weekly notebooks in your solutions to the HW.

## PROBLEM 1: Affine gap global alignment with homopolymer indels (60 points)

### Homopolymer indels
Some cutting-edge DNA sequencing technologies, such as Oxford Nanopore, perform poorly in regions of sequences that are *homopolymers*, which are runs of consecutive and identical bases.  For example, in the DNA sequence `GCTAGCCCCCTATC`, the substring `CCCCC` is a homopolymer of the base `C`.  Some sequencers have difficulty in accurately determining the length of such homopolymer regions.  If the example sequence just given were the true sequence, a sequencer could make a mistake and report a sequence of `GCTAGCCCTATC` (truncating the homopolymer region to three `C`s) or `GCTAGCCCCCCTATC` (expanding the homopolyer region to six `C`s), just to give a few examples of possible errors.

When aligning pairs of DNA sequences (say for the purpose of determining overlapping reads in genome assembly), a homopolymer error in a sequence results in an insertion or deletion in that sequence with respect to the other.  Given the knowledge that homopolymer errors are common in sequences from certain sequencing technologies, we may wish to use an alignment scoring scheme that penalizes gaps resulting from homopolymer errors *to a lesser degree* than other, non-homopolymer gaps.

### An algorithm for pairwise alignment with homopolymer indels
In this problem you will implement the dynamic programming algorithm for global alignment with affine gap penalties that distinguishes between homopolymer and non-homopolymer insertions and uses different penalties for the two cases.  There are many ways in which this could be approached.  We will use one specific approach that considers a gap to be a *homopolymer gap* if the inserted sequence is a homopolymer and the base preceding the inserted sequence is the same as the base within the homopolymer.  Note that in pairwise alignment, a deletion in one sequence is indistinguishable from an insertion in the second sequence, and therefore we will consider all gaps to be insertions.  We will score a homopolymer gap of length $k$ using the  function:
$$w_h(k) = h + tk$$
and non-homopolymer gaps with the function
$$w(k) = g + sk,$$
and we will generally have that $h > g$ and $t > s$ such that homopolymer gaps penalized less than non-homopolymer gaps (note that all parameters are negative).

It turns out that we can find an optimal alignment with this scoring scheme using a modification of the standard affine gap global alignment dynamic programming algorithm.  Specifically, we will introduce two additional matrices, $H_x$ and $H_y$, which will keep track of alignments that end with a homopolymer insertion in $x$ and $y$, respectively.  To be precise, $H_x[i, j]$ will be defined as the optimal score of an alignment of the first $i$ characters of $x$ and the first $j$ characters of $y$ that ends in a homopolymer insertion in $x$.  The $H_x$ and $H_y$ matrices are similar to the $I_x$ and $I_y$ matrices in meaning, but are only for alignments ending in homopolymer gaps.  The dynamic programming recurrences thus become:

$M(i, j) = \max\left\{
\begin{array}{l}
M(i - 1, j - 1) + S(x_i, y_j) \\
I_x(i - 1, j - 1) + S(x_i, y_j) \\
I_y(i - 1, j - 1) + S(x_i, y_j) \\
H_x(i - 1, j - 1) + S(x_i, y_j) \\
H_y(i - 1, j - 1) + S(x_i, y_j) \\
\end{array}
\right.$

$I_x(i, j) = \max\left\{
\begin{array}{l}
M(i - 1, j) + g + s \\
I_x(i - 1, j) + s \\
\end{array}
\right.$

$I_y(i, j) = \max\left\{
\begin{array}{l}
M(i, j - 1) + g + s \\
I_y(i, j - 1) + s \\
\end{array}
\right.$

$H_x(i, j) = \delta(x[i] \neq x[i - 1]) + 
\max\left\{
\begin{array}{l}
M(i - 1, j) + h + s \\
H_x(i - 1, j) + s \\
\end{array}
\right.$

$H_y(i, j) = \delta(y[j] \neq y[j - 1]) +
\max\left\{
\begin{array}{l}
M(i, j - 1) + h + s \\
H_y(i, j - 1) + s \\
\end{array}
\right.$

In the recurrences for $H_x$ and $H_y$, the function $\delta$, is defined as 
$$\delta(condition) = \left\{
\begin{array}{ll}
-\infty, &\textrm{if}\ condition \\
0, & \textrm{otherwise}
\end{array}
\right.$$
which serves to restrict the scores in the $H$ matrices to those of alignments that end in homopolymer indels.

Note that the definition of a homopolymer gap provided above does *not* include insertions of a homopolymer at the beginning of either sequence, since there would be no base preceding such an insertion.  Therefore, for initialization, the entries of the first row and column of both the $H_x$ and $H_y$ matrices are set to $-\infty$.

### Your task
Implement the algorithm described above as a function `align_global_affine_hp_gaps` below, that takes as input two sequences, `x` and `y`, a substitution matrix, the gap scoring parameters ($g$, $h$, $s$, and $t$).  The substitution matrix will be represented as a dictionary with two-element tuples, `(a, b)`, as keys and scores as values.  Your function should output a tuple of two elements, the first being the score of an optimal alignment, and the second being a single alignment that obtains that score. Your alignment should be represented as a list of two strings.  See the "Tests for PROBLEM 1" section at the bottom of this notebook for examples of the inputs and outputs.  

In the case that there are multiple optimal alignments, during the traceback, if there are ties for which matrix to jump back to at each step, the order of preference for which matrix to jump to should be ($I_x$, $H_x$, $M$, $I_y$, $H_y$).  For example, in a $H_y$ cell, if the $M$ and $H_y$ cases in the recurrence both tie for the maximum, then during the traceback, you should prefer to jump back to the $M$ cell.

Your implementation must use an efficient (polynomial-time) dynamic programming algorithm (i.e., either top-down or bottom-up).

In [79]:
# Code for PROBLEM 1
# You are welcome to develop your code as a separate Python module
# and import it here if that is more convenient for you.

# You may find the following constant value for negative infinity of use
NEGATIVE_INFINITY = float("-inf")


def align_global_affine_hp_gaps(x, y, submatrix, g, h, s, t):
    """Computes an optimal global pairwise alignment 
    with an affine gap homopolymer-aware scoring function.
        
    Args:
        x: a string representing the first sequence
        y: a string representing the second sequence
        submatrix: a substitution matrix
        g: the gap existence score for non-homopolymer gaps
        h: the gap existence score for homopolymer gaps
        s: the space score for non-homopolymer gaps
        t: the space score for homopolymer gaps
    Returns:
        A tuple, (score, alignment), where score is a numeric value giving the score of the
        alignment and alignment is a list of two strings
    """
    def print_matrix(F):
        for i in range(imax + 1):
            for j in range(jmax + 1):
                print(F(i, j), end="\t")
            print()
        print()
        
    def memoize(func):
        outputs = dict()
        def memoized(arg1, arg2):
            if (arg1, arg2) in outputs:
                return outputs[(arg1, arg2)]
            outputs[(arg1, arg2)] = func(arg1, arg2)
            return outputs[(arg1, arg2)]
        return memoized
    
    imax = len(x)
    jmax = len(y)
    arrows = {}
    
    @memoize
    def M(i, j):
        if i == 0 and j == 0:
            return 0
        if i == 0 or j == 0:
            return NEGATIVE_INFINITY
        score = submatrix[(x[i - 1], y[j - 1])]
        values = [
            Ix(i - 1, j - 1) + score,
            Hx(i - 1, j - 1) + score,
            M(i - 1, j - 1) + score,
            Iy(i - 1, j - 1) + score,
            Hy(i - 1, j - 1) + score,
        ]
        
        max_val = max(values)
        max_idx = values.index(max_val)
        max_mat = ["Ix", "Hx", "M", "Iy", "Hy"][max_idx]
        
        arrows[("M", i, j)] = (max_mat, i - 1, j - 1)
    
        return max_val

    @memoize
    def Ix(i, j):
        if i == 0 and j == 0:
            return g
        if i == 0:
            return NEGATIVE_INFINITY
        values = [
            Ix(i - 1, j) + s,
            M(i - 1, j) + g + s,
        ]
        
        max_val = max(values)
        max_idx = values.index(max_val)
        max_mat = ["Ix", "M"][max_idx]
        
        arrows[("Ix", i, j)] = (max_mat, i - 1, j)
    
        return max_val

    @memoize
    def Iy(i, j):
        if i == 0 and j == 0:
            return g
        if j == 0:
            return NEGATIVE_INFINITY
        values = [
            M(i, j - 1) + g + s,
            Iy(i, j - 1) + s,
        ]
        
        max_val = max(values)
        max_idx = values.index(max_val)
        max_mat = ["M", "Iy"][max_idx]
        
        arrows[("Iy", i, j)] = (max_mat, i, j - 1)
    
        return max_val
        
    
    @memoize
    def Hx(i, j):
        if i == 0 or j == 0:
            return NEGATIVE_INFINITY
        if x[i - 1] != x[i - 2]:
            return NEGATIVE_INFINITY
        values = [
            Hx(i - 1, j) + t,
            M(i - 1, j) + h + t,
        ]
        
        max_val = max(values)
        max_idx = values.index(max_val)
        max_mat = ["Hx", "M"][max_idx]
        
        arrows[("Hx", i, j)] = (max_mat, i - 1, j)
        
        return max_val
        
    
    @memoize
    def Hy(i, j):
        if i == 0 or j == 0:
            return NEGATIVE_INFINITY
        if y[j - 1] != y[j - 2]:
            return NEGATIVE_INFINITY
        values = [
            M(i, j - 1) + h + t,
            Hy(i, j - 1) + t,
        ]
    
        max_val = max(values)
        max_idx = values.index(max_val)
        max_mat = ["M", "Hy"][max_idx]
        
        arrows[("Hy", i, j)] = (max_mat, i, j - 1)
    
        return max_val
            
    values = [
        Ix(imax, jmax),
        Hx(imax, jmax),
        M(imax, jmax),
        Iy(imax, jmax),
        Hy(imax, jmax),
    ]
    
    max_val = max(values)
    max_idx = values.index(max_val)
    max_mat = ["Ix", "Hx", "M", "Iy", "Hy"][max_idx]
    
    path = [(max_mat, imax, jmax)]
    while path[-1] in arrows:
        path.append(arrows[path[-1]])
    
    result_x, result_y = "", ""
    x_idx, y_idx = 0, 0
    for op, _, _ in reversed(path[:-1]):
        if op == "M":
            result_x += x[x_idx]
            result_y += y[y_idx]
            x_idx += 1
            y_idx += 1
        elif op == "Ix" or op == "Hx":
            result_x += x[x_idx]
            result_y += "-"
            x_idx += 1
        elif op == "Iy" or op == "Hy":
            result_x += "-"
            result_y += y[y_idx]
            y_idx += 1
    
#     print(path)
#     print_matrix(M)
#     print_matrix(Ix)
#     print_matrix(Iy)
#     print_matrix(Hx)
#     print_matrix(Hy)
    
    return max_val, [result_x, result_y]

## PROBLEM 2: A strange optimizing ribosome (20 POINTS)

Recall that the ribosomes are the molecular machines (which are actually complexes of proteins and RNAs) that perform the translation of RNA to protein.  Usually, the ribosome begins translation at a start codon and then translates each immediately following codon in turn until a stop codon is reached.  Suppose that we encounter a very strange ribosome that occassionally skips over some number of bases after it translates a codon, resulting in one or more codons being skipped and/or the reading frame changing.  For example in the scenario below, the strange ribosome skips the first two bases, then translates a codon, then skips one base, and then translates two consecutive codons.

![strange ribosome](hw2_strange_ribosome.png)

It appears that this strange ribosome is optimizing an objective function that rewards the translation of specific amino acids and penalizes for the number of bases that are skipped. Letting $\mathbf{x}$ denote an RNA sequence and $\mathbf{C}$ denote the set of start positions of the (non-overlapping) codons that are translated by the ribosome, the ribosome *maximizes* the objective function:

$f(\mathbf{C}) = s(|\mathbf{x}| - 3|\mathbf{C}|) + \sum_{i \in C} r\left(translation\left(x_i x_{i+1} x_{i+2}\right)\right)$

where $r(a)$ denotes the reward that the ribosome gets for translating amino acid $a$, $s$ denotes the (negative) penalty that the ribosome incurs for skipping a base (which includes bases skipped at the beginning of the RNA, at the end of the RNA, and in between translated codons), and $translation$ denotes the function that translates a codon sequence to its corresponding amino acid.

For example, with $s = -1$ and 

$r(a) = \left\{\begin{array}{ll}
6 & \textrm{if } a < \textrm{'M'} \\
1 & \textrm{if } a \geq \textrm{'M'} \\
-\infty & \textrm{if } a \textrm{ is the stop codon symbol}
\end{array}\right.$

where $a < \textrm{'M'}$ indicates that the character for the amino acid $a$ is lexigraphically smaller than 'M', 
the value of the objective function for the set of codons translated by strange ribosome in the figure above is 

$\begin{eqnarray}
f(\{3, 7, 10\}) & = & s(12 - 3 \cdot 3) + r(\textrm{G}) + r(\textrm{H}) + r(\textrm{R}) \\
& = & -3 + 6 + 6 + 1 \\
& = & 10
\end{eqnarray}
$

In this problem, we will figure out how to compute the set of codons that is translated by the strange ribosome for any sequence, $\mathbf{x}$, by using a dynamic programming algorithm to maximize the objective function.  Let a subproblem $F[i]$ denote the score of the best set of codon start positions for the prefix of length $i$ of $\mathbf{x}$.

**(A)** Give initialization equations for $F[0]$, $F[1]$, and $F[2]$.

**(B)** To devise a recurrence equation for $F[i]$, for $i > 2$, we consider two possible cases for the best set of codon start positions for the $i$th prefix: either (i) the last three bases of the prefix form a codon or (ii) the last three bases do not form a codon (which implies that the last base of the prefix is not in a codon).  Fill in the blanks in the recurrence equation below.

$F[i] = \max\left\{
\begin{array}{l}
F[i - 3] + \underline{\hspace{2in}} \\
F[i - 1] + \underline{\hspace{2in}}
\end{array}
\right.$

**(C)** What amino acid sequence would this strange ribosome produce for the RNA sequence `CGCCCAAGGUUG` with the example scoring scheme given above?  Show your work.

#################################################################################

**(A)**

$F[0] = 0$

$F[1] = s$

$F[2] = 2s$

**(B)**

$F[i] = \max\left\{
\begin{array}{l}
F[i - 3] + r\left(translation\left(x_{i-2} x_{i-1} x_{i}\right)\right) \\
F[i - 1] + s
\end{array}
\right.$


**(C)**

$F[0] = 0$

$F[1] = -1$

$F[2] = -2$

$
F[3] = \max\left\{ 
\begin{array}{l} 
F[0] + r(translate(CGC)) = F[0] + r(R) = 0 + 1 = 1 \\ 
F[2] + s = -2 + (-1) = -3 
\end{array}
\right.
= 1
$

$
F[4] = \max\left\{ 
\begin{array}{l} 
F[1] + r(translate(GCC)) = F[1] + r(A) = -1 + 6 = 5 \\ 
F[3] + s = 1 + (-1) = 0 
\end{array}
\right.
= 5
$

$
F[5] = \max\left\{ 
\begin{array}{l} 
F[2] + r(translate(CCC)) = F[2] + r(P) = -2 + 1 = -1 \\ 
F[4] + s = 5 + (-1) = 4 
\end{array}
\right.
= 4
$

$
F[6] = \max\left\{ 
\begin{array}{l} 
F[3] + r(translate(CCA)) = F[3] + r(P) = 1 + 1 = 2 \\ 
F[5] + s = 4 + (-1) = 3 
\end{array}
\right.
= 3
$

$
F[7] = \max\left\{ 
\begin{array}{l} 
F[4] + r(translate(CAA)) = F[4] + r(Q) = 5 + 1 = 6 \\ 
F[6] + s = 3 + (-1) = 2 
\end{array}
\right.
= 6
$

$
F[8] = \max\left\{ 
\begin{array}{l} 
F[5] + r(translate(AAG)) = F[5] + r(K) = 4 + 6 = 10 \\ 
F[7] + s = 6 + (-1) = 5 
\end{array}
\right.
= 10
$

$
F[9] = \max\left\{ 
\begin{array}{l} 
F[6] + r(translate(AGG)) = F[6] + r(R) = 3 + 1 = 4 \\ 
F[8] + s = 10 + (-1) = 9 
\end{array}
\right.
= 9
$

$
F[10] = \max\left\{ 
\begin{array}{l} 
F[7] + r(translate(GGU)) = F[7] + r(G) = 6 + 6 = 12 \\ 
F[9] + s = 9 + (-1) = 8 
\end{array}
\right.
= 12
$

$
F[11] = \max\left\{ 
\begin{array}{l} 
F[8] + r(translate(GUU)) = F[8] + r(V) = 10 + 1 = 11 \\ 
F[10] + s = 12 + (-1) = 11 
\end{array}
\right.
= 11
$

$
F[12] = \max\left\{ 
\begin{array}{l} 
F[9] + r(translate(UUG)) = F[9] + r(L) = 9 + 6 = 15 \\ 
F[11] + s = 11 + (-1) = 10 
\end{array}
\right.
= 15
$

Tracing back the recurrence relation for $F[12]$, we can find that the DNA sequence is recognized as "C GCC C AAG G UUG", which corrseponds to the amino acid sequence "AKL"


#################################################################################

In [82]:
translation = {
 'AAA': 'K', 'AAC': 'N', 'AAG': 'K', 'AAU': 'N',
 'ACA': 'U', 'ACC': 'U', 'ACG': 'U', 'ACU': 'U',
 'AGA': 'R', 'AGC': 'S', 'AGG': 'R', 'AGU': 'S',
 'AUA': 'I', 'AUC': 'I', 'AUG': 'M', 'AUU': 'I',
 'CAA': 'Q', 'CAC': 'H', 'CAG': 'Q', 'CAU': 'H',
 'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCU': 'P',
 'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGU': 'R',
 'CUA': 'L', 'CUC': 'L', 'CUG': 'L', 'CUU': 'L',
 'GAA': 'E', 'GAC': 'D', 'GAG': 'E', 'GAU': 'D',
 'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCU': 'A',
 'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGU': 'G',
 'GUA': 'V', 'GUC': 'V', 'GUG': 'V', 'GUU': 'V',
 'UAA': '*', 'UAC': 'Y', 'UAG': '*', 'UAU': 'Y',
 'UCA': 'S', 'UCC': 'S', 'UCG': 'S', 'UCU': 'S',
 'UGA': '*', 'UGC': 'C', 'UGG': 'W', 'UGU': 'C',
 'UUA': 'L', 'UUC': 'F', 'UUG': 'L', 'UUU': 'F',
}

def memoize(func):
    outputs = dict()
    def memoized(arg):
        if arg not in outputs:
            outputs[arg] = func(arg)
        return outputs[arg]
    return memoized

s = -1
seq = "CGCCCAAGGUUG"

def r(a):
    if a == "*":
        return float("-inf")
    if a < "M":
        return 6
    if a >= "M":
        return 1
    assert False

@memoize
def F(i):
    if i <= 2:
        return i * s
    codon = seq[i-3:i]
    genetic_code = translation[codon]
    F3 = F(i - 3)
    F1 = F(i - 1)
    r_score = r(genetic_code)
    option1 = F3 + r_score
    option2 = F1 + s
    result = max(option1, option2)
    print(
        f"$\n"
        f"F[{i}] = \max\left\{{ \n"
        f"\\begin{{array}}{{l}} \n"
        f"F[{i - 3}] + r(translate({codon})) = F[{i-3}] + r({genetic_code}) = {F3} + {r_score} = {option1} \\\\ \n"
        f"F[{i - 1}] + s = {F1} + ({s}) = {option2} \n"
        f"\\end{{array}}\n"
        f"\\right.\n"
        f"= {result}\n"
        f"$\n"
    )
    return result

# F(12)

## PROBLEM 3: Classifying bacterial DNA fragments (20 POINTS)

Suppose that we obtain some microbial DNA from a sample of water taken from Lake Mendota and sequence a number of short DNA fragments within this sample.  A common task in such experiments is to identify the microbial species to which each DNA fragment belongs.  Suppose that one (very short) sequence obtained in this experiment is:

`GTGATCTTCA`

which has 2 `A`s, 2 `C`s, 2 `G`s, and 4 `T`s.  

Further, suppose that there are only three microbial species, A, B, and C, to which the sequence could belong.  Unfortunately, we do not have the genome sequences for these species, but we do know the overall frequencies of bases within the genomes of the three species, which are given in the table below:

Species | % A | % C | % G | % T
--- | --- | ---
A | 25 | 25 | 25 | 25
B | 30 | 20 | 20 | 30
C | 10 | 40 | 40 | 10

In addition, we know that the relative abundances of the three species within Lake Mendota are:

Species | % abundance
--- | --- 
A | 50
B | 25
C | 25

Let $X$ be a random variable representing the vector of *counts* of the bases within a sequence fragment, and let $Y$ be a random variable representing the species to which that sequence belongs.  Let $x = [2, 2, 2, 4]$ be the vector of counts of `A`, `C`, `G`, and `T`, respectively, in the sequence above.  Compute the following and *show your work*:

**(A)** $P(X = x\ |\ Y = A)$

**(B)** $P(X = x\ |\ Y = B)$

**(C)** $P(X = x\ |\ Y = C)$

**(D)** $P(X = x)$

**(E)** $P(Y = A\ |\ X = x)$

#################################################################################


**(A)** 

$$
\begin{align}
P(X = x\ |\ Y = A) = \frac{(2+2+2+4)!}{2! \times 2! \times 2! \times 4!} \times (25\%)^2 \times (25\%)^2 \times (25\%)^2 \times (25\%)^4 = 0.018024
\end{align}
$$

**(B)** 

$$
\begin{align}
P(X = x\ |\ Y = B) = \frac{(2+2+2+4)!}{2! \times 2! \times 2! \times 4!} \times (30\%)^2 \times (20\%)^2 \times (20\%)^2 \times (30\%)^4 = 0.022044
\end{align}
$$
 
**(C)** 

$$
\begin{align}
P(X = x\ |\ Y = C) & = \frac{(2+2+2+4)!}{2! \times 2! \times 2! \times 4!} \times (10\%)^2 \times (40\%)^2 \times (40\%)^2 \times (10\%)^4 = 0.00048384
\end{align}
$$
 
**(D)** 

$$
\begin{align}
P(X = x) & =  P(X = x\ |\ Y = A)P(Y = A) + P(X = x\ |\ Y = B)P(Y = B) + P(X = x\ |\ Y = C)P(Y = C) \\ 
& = 0.018024 \times 50\% +  0.022044 \times 25\% + 0.00048384 \times 25\% \\ 
& = 0.01464396
\end{align}
$$
 
**(E)** 

$$
\begin{align}
P(Y = A\ |\ X = x) & = \frac{P(X = x\ |\ Y = A)P(Y = A)}{P(X = x)} \\
& = \frac{0.018024\times 50\%}{0.01464396} \\
& = 0.615407308
\end{align}
$$


#################################################################################

## Tests for PROBLEM 1

### Substitution matrices for testing

In [83]:
DNA = "ACGT"
BASE_TYPE = {'A': 'purine',
             'G': 'purine',
             'C': 'pyrimidine',
             'T': 'pyrimidine'}

# A simple match=+10 and mismatch=-10 substitution matrix
basic_submatrix = {(a, b): 10 if a == b else -10 for a in DNA for b in DNA}

# A matrix that penalizes transitions less than transversions
tt_submatrix = {(a, b): 10 if a == b else (-10 if BASE_TYPE[a] != BASE_TYPE[b] else -5)
                for a in DNA for b in DNA}

# A utility function for displaying a substitution matrix
def print_dict_matrix(m, width=5):
    """Prints the dict-based matrix m with the specified width for each column."""
    def print_row(fields):
        print("".join(["{:>{width}}".format(field, width=width) for field in fields]))
    labels = sorted({c for pair in m for c in pair})
    print_row([""] + labels)
    for a in labels:
        print_row([a] + [m.get((a, b), "") for b in labels])

### Functions for scoring an a global alignment with affine and context-sensitive gap scores

In [84]:
def score_alignment(alignment, submatrix, g, h, s, t):
    """Returns the score of the alignment given a substitution matrix and gap scores."""
    aligned_pair_scores = [submatrix[pair] for pair in zip(*alignment) if '-' not in pair]
    insert_scores = list(insertion_scores(alignment, g, h, s, t))
    return sum(aligned_pair_scores) + sum(insert_scores)
    
def insertion_scores(alignment, g, h, s, t):
    """Returns a list of the scores of the insertions within the alignment."""
    for i, start, end in insertions(alignment):
        hp_base = alignment[i][start - 1: start]
        length = end - start
        if alignment[i][start: end] == hp_base * length:
            yield h + length * t
        else:
            yield g + length * s

def insertions(alignment):
    """Returns an iterator over the insertions within the alignment.
    Each element of the iterator has the form (seq_index, start, end)"""
    for i, seq in enumerate(alignment):
        for gap in gaps(seq):
            yield (1 - i, gap[0], gap[1])

import re
def gaps(s):
    """Returns an iterator over the gaps of the string s.
    Each element of the iterator has the form (start, end)"""
    gap_pattern = re.compile('-+')
    for match in gap_pattern.finditer(s):
        yield (match.start(), match.end())

###  Test cases and testing functions

In [85]:
test_case_inputs = {
    'small_1':  ("AGTA", "AGA",  basic_submatrix, -20, -5, -10, -1),
    'small_2':  ("AGT",  "AGT",  basic_submatrix, -20, -5, -10, -1),
    'small_3':  ("AGT",  "G",    basic_submatrix, -20, -5, -10, -1),
    'small_4':  ("G",    "AGT",  basic_submatrix, -20, -5, -10, -1),
    'small_5':  ("AGT",  "",     basic_submatrix, -20, -5, -10, -1),
    'small_6':  ("",     "AGT",  basic_submatrix, -20, -5, -10, -1),
    'small_7':  ("A",    "",     basic_submatrix, -20, -5, -10, -1),
    'small_8':  ("CCAG", "AGTT", tt_submatrix,    -20, -5, -10, -1),
    'small_9':  ("CTC",  "TGCT", basic_submatrix,   0,  0, -10, -1),
    'small_10': ('TTTA', 'TAAA', basic_submatrix, -20, -5, -10, -1),
    'large_1':  ("GAATCGGTCCATTTCACAGTTCACACGCGAGGATTTTTGACCTCTGCACAAGCGCGCACATCG",
                 "TTTTTTATACCCTCTCTACGCCCGGAATCCACAGAACGACCTACCATCTGGGTTACATCGCGCAGATTTAA",
                 basic_submatrix, -20, -5, -10, -1)
}

test_case_correct_outputs = {
    'small_1': (0,   ['AGTA', 
                      'AG-A']),
    'small_2': (30,  ['AGT', 
                      'AGT']),
    'small_3': (-50, ['AGT', 
                      'G--']),
    'small_4': (-50, ['--G', 
                      'AGT']),
    'small_5': (-50, ['AGT', 
                      '---']),
    'small_6': (-50, ['---', 
                      'AGT']),
    'small_7': (-30, ['A', 
                      '-']),
    'small_8': (-37, ['CCAG-', 
                      'A-GTT']),
    'small_9': (-10, ['--CTC', 
                      'TGCT-']),
    'small_10': (6,  ['TTTA--',
                      'T--AAA']),
    'large_1': (18, ['G-----AATC--GGTCCATTTCACAGT-TC-ACACG-CGAGGATTTTTGACCTCTG--C-ACAAGCGCGCACAT--CG',
                     'TTTTTTATACCCTCTCTACG-CCCGGAATCCACAGAACGAC-CT----ACCATCTGGGTTACA-TCGCGCAGATTTAA'])
}

import numbers
def check_valid_alignment_result(result, x, y):
    """Checks that the alignment result is valid for sequences x and y."""
    assert isinstance(result, tuple), "Output is not a tuple"
    assert len(result) == 2, "Output does not have exactly two elements"
    score, alignment = result
    assert isinstance(alignment, list), "Alignment is not a list"
    assert isinstance(score, numbers.Number), "Score is not a number"
    assert len(alignment) == 2, "Alignment does not have exactly two elements"
    assert all(isinstance(element, str) for element in alignment), "Alignment elements are not strings"
    assert len(alignment[0]) == len(alignment[1]), "Alignment strings do not have the same length"
    assert alignment[0].replace('-', '') == x, "First string of alignment is not x"
    assert alignment[1].replace('-', '') == y, "Second string of alignment is not y"

def check_valid_alignment_score(result, submatrix, g, h, s, t):
    """Checks that the computed score of the alignment is equal to the score given in the result."""
    score, alignment = result
    computed_score = score_alignment(alignment, submatrix, g, h, s, t)
    assert computed_score == score, "Computed score ({}) does not equal the returned score ({})".format(computed_score,
                                                                                                        score)
def check_test_case(case_name, test_name=None, valid_result=True, valid_score=True, correct_alignment=True):
    inputs = test_case_inputs[case_name]
    correct_output = test_case_correct_outputs[case_name]
    result = align_global_affine_hp_gaps(*inputs)
    if valid_result:
        check_valid_alignment_result(result, *inputs[:2])
    if valid_score:
        check_valid_alignment_score(result, *inputs[2:])
    if correct_alignment:
        error_template = ("Incorrect output\n"
                          "returned (score = {0[0]}):\n"
                          "{0[1][0]}\n"
                          "{0[1][1]}\n"
                          "correct (score = {1[0]}):\n"
                          "{1[1][0]}\n"
                          "{1[1][1]}")
        assert result == correct_output, error_template.format(result,
                                                               correct_output)
    print("SUCCESS:", test_name if test_name else case_name, "passed!")

### Visible tests

In [86]:
# TEST: small_1 valid output
check_test_case("small_1", test_name="small_1 valid output", valid_score=False, correct_alignment=False)

SUCCESS: small_1 valid output passed!


In [87]:
# TEST: small_1 valid score
check_test_case("small_1", test_name="small_1 valid score", correct_alignment=False)

SUCCESS: small_1 valid score passed!


In [88]:
# TEST: small_2
check_test_case("small_2")

SUCCESS: small_2 passed!


In [89]:
# TEST: small_3
check_test_case("small_3")

SUCCESS: small_3 passed!


In [90]:
# TEST: small_4
check_test_case("small_4")

SUCCESS: small_4 passed!


In [91]:
# TEST: small_5
check_test_case("small_5")

SUCCESS: small_5 passed!


In [92]:
# TEST: small_6
check_test_case("small_6")

SUCCESS: small_6 passed!


In [93]:
# TEST: small_7
check_test_case("small_7")

SUCCESS: small_7 passed!


In [94]:
# TEST: small_8
check_test_case("small_8")

SUCCESS: small_8 passed!


In [95]:
# TEST: small_9
check_test_case("small_9")

SUCCESS: small_9 passed!


In [96]:
# TEST: small_10
check_test_case("small_10")

SUCCESS: small_10 passed!


In [97]:
# TEST: large_1
check_test_case("large_1")

SUCCESS: large_1 passed!


### Hidden tests

In [98]:
# TEST: large_2
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# TEST: large_3
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# TEST: large_4
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# TEST: large_5
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# TEST: large_6
###
### AUTOGRADER TEST - DO NOT REMOVE
###
